# An example of `epydemic`

This notebook extracts the code from the slides as a basis for experimentation.

A lot of the methods described spearately in the slides have to be put into one cell here, because they're all part of the same class.

Fiorst we impoirt the classes we need. We *don't* import `epydemic`'s SIR class as that's what we're going to be building in this notebook &ndash; you can find the real code [here](https://github.com/simoninireland/epydemic/blob/master/epydemic/sir_model.py).

In [1]:
# simulation support
from epyc import Experiment
from epydemic import ERNetwork, StochasticDynamics, CompartmentedModel

# analysis and plotting
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['font.size'] = 10
import matplotlib.pyplot as plt

The model consists of four parts:

1. Some setup code defining parameter names we need
2. A `build()` method that builds the thigs we need to run the simulation
3. Two evens, `infect()` and `remove()`, that implement the basic discrete events
4. Results-extraction code in `results()`

In [2]:
class SIR(CompartmentedModel):

    # ---------- Setup ----------
    
    # Names for parameters to be passed to experiments
    P_INFECTED = 'epydemic.sir.pInfected'
    P_INFECT = 'epydemic.sir.pInfect'
    P_REMOVE = 'epydemic.sir.pRemove'

    # Names for the compartments, for use in node attributes
    SUSCEPTIBLE = 'epydemic.sir.S'
    INFECTED = 'epydemic.sir.I'
    REMOVED = 'epydemic.sir.R'   
    SI = 'epydemic.sir.SI'

    
    # ---------- Building the simulation model ----------
    
    def build(self, params):
        super().build(params)

        # extract the parameters we need
        pInfected = params[self.P_INFECTED]
        pInfect = params[self.P_INFECT]
        pRemove = params[self.P_REMOVE]

        # create the compartments
        self.addCompartment(self.SUSCEPTIBLE, 1 - pInfected)
        self.addCompartment(self.INFECTED, pInfected)
        self.addCompartment(self.REMOVED, 0.0)

        # declare the sets we need to track
        self.trackEdgesBetweenCompartments(self.SUSCEPTIBLE,
                                           self.INFECTED,
                                           name=self.SI)
        self.trackNodesInCompartment(self.INFECTED)
    
        # add stochastic events to these sets
        self.addEventPerElement(self.SI,
                                pInfect,
                                self.infect,            # defined below
                                name=self.INFECTED)
        self.addEventPerElement(self.INFECTED,
                                pRemove,
                                self.remove,            # defined below
                                name=self.REMOVED)

        
    # ---------- Discrete events ----------
    
    def infect(self, t, e):
       # extract the “S” end of the edge
       (n, _) = e

       # change the “S” to an “I”
       self.changeCompartment(n, self.INFECTED)
        
    def remove(self, t, n):
       # change to an “R"
       self.changeCompartment(n, self.REMOVED)
    
    
    # ---------- Results extraction ----------
    
    def results(self):
        rc = super().results()

        # add the size of each compartment
        for c in self.compartments():
            rc[c] = len(self.compartment(c))

        # return the results
        return rc

To run a simulation we first define the parameter space that sets up the values of the parameters we need in a Python dict.

In [3]:
# create a dict for the experimental parameters
params = dict()

# set the topology for the generated network
params[ERNetwork.N] = int(1e5)
params[ERNetwork.KMEAN] = 20

# set the disease parameters
params[SIR.P_INFECTED] = 0.01
params[SIR.P_INFECT] = 0.02
params[SIR.P_REMOVE] = 0.002

(This sets up a single experiment: later we'll see how to create a parameter space defining a complete *set* of experiments and perfoirming them all with one command.) We can then create and run the experiment:

In [4]:
# create the process and generator
p = SIR()
g = ERNetwork()

# run the experiment
rc = StochasticDynamics(p, g).set(params).run()

Time passes... How much depends on the machine on which you run the experiment, which is inherently sequential.

What we get back when the experiment completes is a *results dict* structured in a particular way, with parameters, results, and metadata. The parameters section records the parameters of the experiment; the results section contains the information we collected in the `results()` method; and the metadata section includes some data collected about how the experiment ran:

In [5]:
rc

{'parameters': {'N': 100000,
  'kmean': 20,
  'epydemic.sir.pInfected': 0.01,
  'epydemic.sir.pInfect': 0.02,
  'epydemic.sir.pRemove': 0.002,
  'topology': 'ER'},
 'metadata': {'epyc.experiment.classname': 'epydemic.stochasticdynamics.StochasticDynamics',
  'epyc.experiment.start_time': datetime.datetime(2023, 10, 10, 9, 54, 33, 149013),
  'epyc.experiment.setup_time': 5.918024,
  'epydemic.monitor.time': 5178.011941032238,
  'epydemic.monitor.events': 198998,
  'epyc.experiment.experiment_time': 68.799738,
  'epyc.experiment.teardown_time': 1.7e-05,
  'epyc.experiment.end_time': datetime.datetime(2023, 10, 10, 9, 55, 47, 866792),
  'epyc.experiment.elapsed_time': 74.71777900000001,
  'epyc.experiment.status': True},
 'results': {'epydemic.sir.S': 0,
  'epydemic.sir.I': 0,
  'epydemic.sir.R': 100000}}

We can interrogate this data to extract the total running time of the expertiment:

In [6]:
print(rc[Experiment.METADATA][Experiment.END_TIME] - rc[Experiment.METADATA][Experiment.START_TIME])

0:01:14.717779


(This was run on a 3.2GHz Intel Core i5-6500: you may get different timings.